In [2]:
!pip install pandas
!pip install html2text
!pip install schedule
!pip install sinling 
!pip install regex
!pip install deep_translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 50.2 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=1f10fc7ab420f71a0b94a6981daead40734960a9f36f037341fb31e76f3e8952
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji
Looking in indexes: https://pypi.or

In [4]:
import requests
from bs4 import BeautifulSoup
import html2text
from urllib.parse import urljoin
import os
import re
import schedule
import time
from datetime import datetime, timedelta
import random
from deep_translator import GoogleTranslator
import pandas as pd
import textwrap


invalid_urls = [
    'tg://', 'tel:', 'qz', 'ru', 'en', 'cn', 'oz', 'ar'
]

unique_links = set()
processed_links = set()

converter = html2text.HTML2Text()
converter.ignore_links = True
converter.ignore_images = True
converter.single_line_break = True


def tokenize_kazakh(text):
    text = str(text)
    sentence_delimiters = ['[.!?]']
    pattern = r'({0})'.format('|'.join(sentence_delimiters))
    sentences = re.split(pattern, text)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    sentences = [re.sub(r'[.!?""]', '', sentence) for sentence in sentences]
    return sentences


def main(url, output_file):
    url_parts = url.split('/')
    if any(x in url_parts for x in invalid_urls):
        print(f'Skipping invalid URL: {url}')
        return

    try:
        response = requests.get(url)
    except requests.exceptions.SSLError as e:
        print(f'SSLError fetching {url}')
        return

    with open(output_file, 'a', encoding='utf-8') as f:
        main_page_response = requests.get(url)
        main_page_content = main_page_response.text

        soup = BeautifulSoup(main_page_content, 'html.parser')

        for link in soup.find_all('a', href=True):
            link_url = link['href']
            absolute_link_url = urljoin(url, link_url)
            url_parts = absolute_link_url.split('/')
            if not any(x in url_parts for x in invalid_urls):
                if absolute_link_url not in unique_links:
                    unique_links.add(absolute_link_url)

        try:
            plain_text_content = converter.handle(main_page_content)
            f.write(plain_text_content)
        except Exception as e:
            print(f'Error fetching main page {url}: {e}')

        for link_url in unique_links:
            if link_url not in processed_links:
                try:
                    response = requests.get(link_url)
                    content = response.text
                    plain_text_content = converter.handle(content)
                    f.write(plain_text_content)
                    processed_links.add(link_url)
                except requests.exceptions.SSLError as e:
                    print(f'SSLError fetching {link_url}: {e}')
                    continue
                except Exception as e:
                    print(f'Error fetching {link_url}: {e}')
                    continue


def create_parallel_corpus(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        text = f.read()

    kazakh_sentences = tokenize_kazakh(text)

    with open(output_file, 'a', encoding='utf-8') as f:
        for sentence in kazakh_sentences:
            f.write(sentence + '\n')

    df = pd.DataFrame(kazakh_sentences, columns=["kazakh_sentence"])
    df.to_csv(output_file, index=False, encoding='utf-8')


def translate_text(text, target_lang):
    sentences = tokenize_kazakh(text)
    split_sentences = []
    for sentence in sentences:
        if len(sentence) >= 5000:
            print(f"Skipping sentence with length {len(sentence)}: {sentence}")
            with open('split_sentences.csv', 'a') as f:
                f.write(sentence + '\n')
            continue

        split_sentences.append(sentence)

    translator = GoogleTranslator(target=target_lang)
    translated_sentences = []
    max_retries = 5
    backoff_factor = 2
    for sentence in split_sentences:
        if len(sentence.strip()) > 10:
            for attempt in range(max_retries):
                try:
                    translation = translator.translate(sentence, source='kazakh', target=target_lang)
                    translated_sentences.append(translation)
                    break
                except Exception as e:
                    if attempt < max_retries - 1:
                        wait_time = backoff_factor ** (attempt + 1)
                        print(f"Error in translation, retrying in {wait_time} seconds: {e}")
                        time.sleep(wait_time)
                    else:
                        print(f"Translation failed after {max_retries} retries: {e}")
                        translated_sentences.append("[TRANSLATION_FAILED]")  # Add a placeholder for failed translations
        else:
            print(f"Skipping too short or empty sentence: {sentence}")

    return " ".join(translated_sentences)

def translate_parallel_corpus(input_file, output_file, target_lang):
    df = pd.read_csv(input_file, encoding='utf-8')

    # Add sentence_id column
    df['sentence_id'] = range(1, len(df) + 1)

    # Initialize existing_df as an empty DataFrame
    existing_df = pd.DataFrame()

    # Translate only new sentences by sentence_id
    try:
        existing_df = pd.read_csv(output_file, encoding='utf-8')
        existing_ids = set(existing_df.sentence_id)
        new_df = df[~df.sentence_id.isin(existing_ids)]
        translations = []
        for kazakh_sentence in new_df.kazakh_sentence:
            translation = translate_text(kazakh_sentence, target_lang)
            translations.append(translation)
        new_df['translated_sentence'] = translations
    except FileNotFoundError:
        new_df = df
        new_df['translated_sentence'] = [translate_text(s, target_lang) for s in df.kazakh_sentence]

    # Append new translations to output file
    try:
        existing_df.to_csv(output_file, mode='a', index=False, header=False, encoding='utf-8')
        new_df.to_csv(output_file, mode='a', index=False, header=False, encoding='utf-8')
    except FileNotFoundError:
        # Output file does not exist yet, so create it 
        df.to_csv(output_file, index=False, encoding='utf-8')


def get_next_run_time():
    now = datetime.now()
    random_offset = random.randint(24, 28) * 3600
    next_run_time = now + timedelta(seconds=random_offset)
    return next_run_time


def execute_main():
    print("Running main function!")
    main_urls = [
         'https://www.inform.kz/kz',
        'https://www.inform.kz/kz/bilik_g21',
        'https://www.inform.kz/kz/aymak_g25',
        'https://www.inform.kz/kz/medicina_t271',
        'https://www.inform.kz/kz/ekonomika_g22',
        'https://www.inform.kz/kz/search?sword=sports',
        'https://www.inform.kz/kz/kogam_g23',
        'https://www.inform.kz/kz/tagayyndau_s24904'
    ]

    output_files = [
         'kazakh_general_news.txt',
        'kazakh_politics.txt',
        'kazakh_region.txt',
        'kazakh_world.txt',
        'kazakh_economy.txt',
        'kazakh_sports.txt',
        'kazakh_society.txt',
        'kazakh_accidents.txt'
    ]

    for url, output_file in zip(main_urls, output_files):
        main(url, output_file)
        unique_links.clear()

    for output_file in output_files:
        parallel_corpus_file = output_file.replace('.txt', '_parallel_corpus.csv')
        create_parallel_corpus(output_file, parallel_corpus_file)
        translated_parallel_corpus_file = output_file.replace('.txt', '_translated_parallel_corpus.csv')
        translate_parallel_corpus(parallel_corpus_file, translated_parallel_corpus_file, 'en')


if __name__ == "__main__":
    execute_main()
    next_run_time = get_next_run_time()
    print(f"Next run time: {next_run_time}")

    while True:
        if datetime.now() >= next_run_time:
            execute_main()
            next_run_time = get_next_run_time()
            print(f"Next run time: {next_run_time}")
        time.sleep(60)  # check every minute

Streaming output truncated to the last 5000 lines.

03
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: 05
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: 05
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: 05
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: Үкімет

30
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: 04
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short or empty sentence: nan
Skipping too short

KeyboardInterrupt: ignored